<a href="https://colab.research.google.com/github/Ar3mis22/FakeNewsDetectorUsingLSTM/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.metrics import confusion_matrix,accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df=pd.read_csv('/content/train.csv',encoding = "ISO-8859-1")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
X.shape


(20800, 4)

In [ ]:
y.shape

(20800,)

In [ ]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=10000

In [ ]:
messages=X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3454 3367 4724]
 [   0    0    0 ... 1053 9702  866]
 [   0    0    0 ... 8248 2259 4132]
 ...
 [   0    0    0 ... 9292 9305 7655]
 [   0    0    0 ... 2896 5146 8147]
 [   0    0    0 ... 3686 3163 8185]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2810,
       9550, 2793, 1790, 8260, 5817, 8627, 3454, 3367, 4724], dtype=int32)

In [ ]:
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           1000000   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1080501 (4.12 MB)
Trainable params: 1080501 (4.12 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(20800, (20800,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((20800, 20), (20800,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 15s 57ms/step - loss: 0.2641 - accuracy: 0.8802 - val_loss: 0.1783 - val_accuracy: 0.9270
Epoch 2/10
218/218 [==============================] - 12s 53ms/step - loss: 0.1040 - accuracy: 0.9612 - val_loss: 0.1865 - val_accuracy: 0.9270
Epoch 3/10
218/218 [==============================] - 12s 54ms/step - loss: 0.0628 - accuracy: 0.9782 - val_loss: 0.2191 - val_accuracy: 0.9229
Epoch 4/10
218/218 [==============================] - 12s 54ms/step - loss: 0.0337 - accuracy: 0.9894 - val_loss: 0.2602 - val_accuracy: 0.9171
Epoch 5/10
218/218 [==============================] - 12s 54ms/step - loss: 0.0176 - accuracy: 0.9951 - val_loss: 0.3131 - val_accuracy: 0.9203
Epoch 6/10
218/218 [==============================] - 12s 56ms/step - loss: 0.0098 - accuracy: 0.9968 - val_loss: 0.3076 - val_accuracy: 0.9263
Epoch 7/10
218/218 [==============================] - 12s 53ms/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.4032 - val_accuracy:

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

215/215 [==============================] - 2s 9ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3154,  295],
       [ 251, 3164]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9204545454545454